# 预处理

In [3]:
import pandas as pd
import os

# 参数设置
# 设置输入和输出文件夹路径
input_folder = r'E:\00DIC全球\ERA5数据'  # 替换为你的主文件夹路径
output_folder = r'E:\00DIC全球\ERA5数据'    # 替换为输出文件夹路径
year_range = range(1984, 2025)  # 包含1984-2024

def process_data():
    # 创建数据结构：{年份: {HYBAS_ID: {属性: 值}}}
    year_dict = {year: {} for year in year_range}
    
    # 遍历所有CSV文件
    for root, _, files in os.walk(input_folder):
        for file in files:
            if not file.lower().endswith('.csv'):
                continue
            
            # 从文件路径提取属性名（示例：用文件名作为属性）
            attr_name = os.path.splitext(file)[0]  # 例如 "温度.csv" -> "温度"
            
            # 读取CSV数据
            df = pd.read_csv(os.path.join(root, file))
            hybas_ids = df.iloc[:, 0]  # 第一列为HYBAS_ID
            
            # 处理每个年份列
            for year_col in df.columns[1:]:
                try:
                    year = int(year_col)
                    if year not in year_range:
                        continue
                except ValueError:
                    continue  # 跳过非年份列
                
                # 更新数据结构
                for hybas_id, value in zip(hybas_ids, df[year_col]):
                    if hybas_id not in year_dict[year]:
                        year_dict[year][hybas_id] = {}
                    year_dict[year][hybas_id][attr_name] = value

    # 保存结果
    os.makedirs(output_folder, exist_ok=True)
    for year in year_range:
        year_data = year_dict[year]
        if not year_data:
            continue
        
        # 转换为DataFrame
        df = pd.DataFrame.from_dict(year_data, orient='index')
        df.index.name = 'HYBAS_ID'
        df.reset_index(inplace=True)
        
        # 保存CSV
        output_path = os.path.join(output_folder, f"{year}.csv")
        df.to_csv(output_path, index=False)
        print(f"已生成：{output_path} (包含{len(df.columns)-1}个属性)")

if __name__ == "__main__":
    process_data()

已生成：E:\00DIC全球\ERA5数据\1984.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1985.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1986.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1987.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1988.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1989.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1990.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1991.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1992.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1993.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1994.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1995.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1996.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1997.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1998.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\1999.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2000.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2001.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2002.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2003.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2004.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2005.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2006.csv (包含11个属性)
已生成：E:\00DIC全球\ERA5数据\2007.csv (包含11个属性)
已生成：E:\00DIC全球\E

In [5]:
import pandas as pd
import os

# ======================
# 配置参数
# ======================
annual_folder = r"E:\00DIC全球\ERA5数据\年均属性值"  # 存储年度CSV的文件夹路径
static_file = r"E:\00DIC全球\ERA5数据\HYBAS_DEM_Slope_Area_Export.csv"  # 固有属性文件路径
output_folder = "path/to/output_folder"  # 输出目录（可与输入目录相同）
merge_suffix = ""  # 合并后文件名后缀（设为空字符串可覆盖原文件）

# ======================
# 函数定义
# ======================
def validate_static_data(static_df):
    """验证固有属性文件结构"""
    if 'HYBAS_ID' not in static_df.columns:
        raise ValueError("固有属性文件必须包含HYBAS_ID列")
    if static_df['HYBAS_ID'].duplicated().any():
        raise ValueError("固有属性文件中存在重复的HYBAS_ID")

def merge_attributes(annual_file, static_df):
    """合并单个年度文件"""
    # 读取年度文件
    annual_path = os.path.join(annual_folder, annual_file)
    df_annual = pd.read_csv(annual_path)
    
    # 检查HYBAS_ID列是否存在
    if 'HYBAS_ID' not in df_annual.columns:
        raise KeyError(f"{annual_file} 中缺少HYBAS_ID列")
    
    # 执行数据合并（左连接）
    merged_df = pd.merge(
        df_annual,
        static_df,
        on='HYBAS_ID',
        how='left',  # 保留所有年度数据中的记录
        validate='m:1'  # 确保静态数据中的HYBAS_ID唯一
    )
    
    # 调整列顺序：静态属性在前
    static_cols = [col for col in static_df.columns if col != 'HYBAS_ID']
    ordered_cols = ['HYBAS_ID'] + static_cols + \
                  [col for col in merged_df.columns 
                   if col not in ['HYBAS_ID'] + static_cols]
    
    return merged_df[ordered_cols]

# ======================
# 主程序
# ======================
if __name__ == "__main__":
    try:
        # 创建输出目录
        os.makedirs(output_folder, exist_ok=True)

        # 步骤1：加载并验证固有属性数据
        static_df = pd.read_csv(static_file)
        validate_static_data(static_df)
        print(f"✅ 成功加载固有属性文件，包含 {len(static_df)} 个唯一HYBAS_ID")

        # 步骤2：处理年度文件
        processed = 0
        annual_files = [f for f in os.listdir(annual_folder) if f.endswith('.csv')]
        
        for file in annual_files:
            try:
                # 执行合并操作
                result_df = merge_attributes(file, static_df)
                
                # 生成输出文件名
                base_name = os.path.splitext(file)[0]
                output_name = f"{base_name}{merge_suffix}.csv"
                output_path = os.path.join(output_folder, output_name)
                
                # 保存结果
                result_df.to_csv(output_path, index=False)
                processed += 1
                print(f"▏ 已处理 {file} → {output_name} | 总记录数: {len(result_df)}")

            except Exception as e:
                print(f"⚠️ 处理 {file} 时出错: {str(e)}")
                continue

        # 输出统计信息
        print(f"\n处理完成！成功合并 {processed}/{len(annual_files)} 个文件")
        print(f"输出目录：{output_folder}")

    except Exception as e:
        print(f"❌ 程序运行失败: {str(e)}")

✅ 成功加载固有属性文件，包含 16397 个唯一HYBAS_ID
▏ 已处理 1984.csv → 1984.csv | 总记录数: 16397
▏ 已处理 1985.csv → 1985.csv | 总记录数: 16397
▏ 已处理 1986.csv → 1986.csv | 总记录数: 16397
▏ 已处理 1987.csv → 1987.csv | 总记录数: 16397
▏ 已处理 1988.csv → 1988.csv | 总记录数: 16397
▏ 已处理 1989.csv → 1989.csv | 总记录数: 16397
▏ 已处理 1990.csv → 1990.csv | 总记录数: 16397
▏ 已处理 1991.csv → 1991.csv | 总记录数: 16397
▏ 已处理 1992.csv → 1992.csv | 总记录数: 16397
▏ 已处理 1993.csv → 1993.csv | 总记录数: 16397
▏ 已处理 1994.csv → 1994.csv | 总记录数: 16397
▏ 已处理 1995.csv → 1995.csv | 总记录数: 16397
▏ 已处理 1996.csv → 1996.csv | 总记录数: 16397
▏ 已处理 1997.csv → 1997.csv | 总记录数: 16397
▏ 已处理 1998.csv → 1998.csv | 总记录数: 16397
▏ 已处理 1999.csv → 1999.csv | 总记录数: 16397
▏ 已处理 2000.csv → 2000.csv | 总记录数: 16397
▏ 已处理 2001.csv → 2001.csv | 总记录数: 16397
▏ 已处理 2002.csv → 2002.csv | 总记录数: 16397
▏ 已处理 2003.csv → 2003.csv | 总记录数: 16397
▏ 已处理 2004.csv → 2004.csv | 总记录数: 16397
▏ 已处理 2005.csv → 2005.csv | 总记录数: 16397
▏ 已处理 2006.csv → 2006.csv | 总记录数: 16397
▏ 已处理 2007.csv → 2007.csv | 总记录数: 16397
▏ 已处理 

# 预测

In [15]:
from joblib import dump, load
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 从本地加载模型（使用原始字符串避免转义问题）
pipeline = load(r'E:\00DIC全球\数据\训练数据\model_predictions_wtDEM.joblib')

    # 数据加载
for year in range(1985,2025):
    # 修正f-string语法，使用原始字符串前缀
    data = pd.read_csv(rf"E:\00DIC全球\ERA5数据\年均属性值\{year}.csv")
    X = data.drop("HYBAS_ID", axis=1)
    ID = data["HYBAS_ID"]

    # 执行预测
    predictions = pipeline.predict(X)

    # 创建结果DataFrame
    result_df = pd.DataFrame({
        'HYBAS_ID': ID,
        'Predicted': predictions
    })

    # 保存预测结果（改为直接保存CSV）
    output_path = rf"E:\00DIC全球\ERA5数据\年均属性值\{year}_预测.csv"
    result_df.to_csv(output_path, index=False)
    print(f"预测结果已保存至：{output_path}")




预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1985_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1986_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1987_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1988_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1989_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1990_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1991_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1992_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1993_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1994_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1995_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1996_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1997_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1998_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\1999_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2000_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2001_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2002_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2003_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2004_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2005_预测.csv
预测结果已保存至：E:\00DIC全球\ERA5数据\年均属性值\2006_预测.csv
预测结果已保存至：E